# Задача по тема 15 (Flask)

Масимум: 3.75т.

Напишете Flask API, който моделира работата със задачи.

Една **задача** съдържа следната информация:
- име (name)
- идентификационен номер (цяло число) (task-id)

Имплементация на задача може да намерите в `task.py` (класът `Task`).

Вашето Flask API трябва да има следните пътища:
- `GET /` - Връща отговор със стойност 200 и тяло "Hello !"
- `POST /task` - Добавя нова задача към системата. 
    - Очаква се, че потребителят ще подаде `name` и `task-id` под формата на JSON в тялото на заявката. (*hint: използвайте `flask.request.json`, за да вземете `dict` с тях*)
    - Ако някой от тези стойности не присъства, върнете отговор с код 400 и тяло: `Not all required arguments are present.`
    - Ако `task-id` не може да бъде взето като цяло число, върнете отговор с код 400 и тяло: `Cannot parse task-id into int. Received {task_id}`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot create task`
    - При успешно добавяне на задача, върнете отговор с код 200 и тяло `Task added successfully`.
- `DELETE /task/<task_id>` - Премахва задача от системата
    - Очаква се, че потребителят ще подаде id-то на желаната за изтриване задача като последната част от пътя на заявката.
    - Ако тази стойност не е цяло число, върнете отговор с код 400 и тяло: `Cannot parse task-id into int. Received {task_id}`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot remove task`
    - Ако търсената задача не съществува, върнете отговор с код 200 и тяло `No task found with id=` и поисканото task_id
    - При успешно премахване на задача, върнете отговор с код 200 и тяло `Task removed successfully`.
- `GET /get/<task_id>` - Връща задача от системата
    - Очаква се, че потребителят ще подаде id-то на желаната задача като последната част от пътя на заявката.
    - Ако тази стойност не е цяло число, върнете отговор с код 400 и тяло: `Cannot parse task-id into int. Received {task_id}`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot read task`
    - Ако търсената задача не съществува, върнете отговор с код 200 и тяло `No task found with id=` и поисканото task_id
    - Ако търсената задача съществува, върнете отговор с код 200 и тяло `{name}`, където `{name}` е името на задачата
- `POST /reset` (вече е имплементирано за вас)
    - Зачиства storage-а на системата
    - Връща отговор с код 200 и тяло `Reset storage succesfully`


Други изисквания, на които трябва вашето API да отговаря:
- Persistent storage - задачите се пазят на външна локация (файл) и не се изгубват при спиране на сървъра. **Използвайте функциите в `storage.py` за тази цел.**
- Файлът за storage трябва да е в JSON формат
- Файлът за storage трябва да се казва `storage.json`
- Файлът за storage трябва да се намира в същата директория, в която е тази тетрадка
- Може да приемете, че няма да се пускат паралелни заявки - т.е. в даден момент от време може да бъде изпълнявана най-много една заявка
- Може да приемете, че няма да има две задачи с еднакви task-id-та
- Преди да изпълните тестовете, стартирайте Flask сървъра ръчно
- Ако адресът на сървъра ви се различава от `http://127.0.0.1:5000`, променете стойноста на променливата в тестовете
- Ако искате да представите непълно решение, закоментирайте тестовете, които не ви работят.

### Тестове и точки:

In [ ]:
import os

import requests

API_URL = 'http://127.0.0.1:5000'

ADD_URL = API_URL + '/task'
REMOVE_URL = API_URL + '/task/{id}'
GET_URL = API_URL + '/task/{id}'
RESET_URL = API_URL + '/reset'

STORAGE_FILE = 'storage.json'

# GET /

def test_01_verify_main_page():
    # Arrange
    expected_code = 200
    expected_text = 'Hello !'

    # Act
    response = requests.get(API_URL)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 1: ✅ 0.25 points")

# POST /task

def test_02_add_task():
    # Arrange
    expected_code = 200
    expected_text = 'Task added successfully'

    body = {'name': 'test_02', 'task-id': '2'}
    requests.post(RESET_URL)

    # Act
    response = requests.post(ADD_URL, json=body)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 2: ✅ 0.25 points")

def test_03_add_task_no_task_id():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required arguments are present'

    body = {'name': 'test_02'}
    requests.post(RESET_URL)

    # Act
    response = requests.post(ADD_URL, json=body)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 3: ✅ 0.25 points")

def test_04_add_task_no_name():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required arguments are present'

    body = {'task-id': '5'}
    requests.post(RESET_URL)

    # Act
    response = requests.post(ADD_URL, json=body)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 4: ✅ 0.25 points")

def test_05_add_task_nan_task_id():
    # Arrange
    expected_code = 400
    task_id = 'notanumber'
    expected_text = f'Cannot parse task_id. Received {task_id}'

    body = {'name': 'foo', 'task-id': task_id}
    requests.post(RESET_URL)

    # Act
    response = requests.post(ADD_URL, json=body)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text in actual_text
    print("Test 5: ✅ 0.25 points")

def test_06_add_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot create task'

    body = {'name': 'foo', 'task-id': '2'}
    requests.post(RESET_URL)
    os.remove(STORAGE_FILE)

    # Act
    response = requests.post(ADD_URL, json=body)
    requests.post(RESET_URL)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 6: ✅ 0.25 points")

def test_07_add_task_existing_id():
    # Arrange
    expected_code = 400
    task_id = '2'
    expected_text = f'Task with id={task_id} already exists'

    body = {'name': 'foo', 'task-id': task_id}
    requests.post(RESET_URL)
    requests.post(ADD_URL, json=body)

    # Act
    response = requests.post(ADD_URL, json=body)
    requests.post(RESET_URL)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text in actual_text
    print("Test 7: ✅ 0.25 points")

# DELETE /task/<id>

def test_08_remove_task():
    # Arrange
    expected_code = 200
    expected_text = 'Task removed successfully'
    task_id = '2'
    
    requests.post(RESET_URL)
    requests.post(ADD_URL, json={'name': 'foo', 'task-id': task_id})

    # Act
    response = requests.delete(REMOVE_URL.format(id=task_id))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 8: ✅ 0.25 points")

def test_09_remove_task_nan_task_id():
    # Arrange
    expected_code = 400
    task_id = 'notanumber'
    expected_text = f'Cannot parse task-id into int. Received {task_id}'

    requests.post(RESET_URL)

    # Act
    response = requests.delete(REMOVE_URL.format(id=task_id))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 9: ✅ 0.25 points")

def test_10_remove_task_non_existing_task():
    # Arrange
    expected_code = 404
    task_id = '3'
    expected_text = f'No task found with id={task_id}'

    requests.post(RESET_URL)
    requests.post(ADD_URL, json={'name': 'foo', 'task-id': '2'})

    # Act
    response = requests.delete(REMOVE_URL.format(id=task_id))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 10: ✅ 0.25 points")

def test_11_remove_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot remove task'

    requests.post(RESET_URL)
    os.remove(STORAGE_FILE)

    # Act
    response = requests.delete(REMOVE_URL.format(id='2'))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 11: ✅ 0.25 points")

# GET /task/<id>

def test_12_get_task():
    # Arrange
    expected_code = 200
    expected_text = 'foo2'

    requests.post(RESET_URL)
    requests.post(ADD_URL, json={'name': 'foo', 'task-id': '2'})
    requests.post(ADD_URL, json={'name': 'foo2', 'task-id': '3'})
    requests.post(ADD_URL, json={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL.format(id='3'))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 12: ✅ 0.25 points")

def test_13_get_task_nan_task_id():
    # Arrange
    expected_code = 400
    task_id = 'notanumber'
    expected_text = f'Cannot parse task_id into int. Received {task_id}'

    requests.post(RESET_URL)
    requests.post(ADD_URL, json={'name': 'foo', 'task-id': '2'})
    requests.post(ADD_URL, json={'name': 'foo2', 'task-id': '3'})
    requests.post(ADD_URL, json={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL.format(id=task_id))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 13: ✅ 0.25 points")

def test_14_get_task_non_existing_task():
    # Arrange
    expected_code = 404
    task_id = '42'
    expected_text = f'No task found with id={task_id}'

    requests.post(RESET_URL)
    requests.post(ADD_URL, json={'name': 'foo', 'task-id': '2'})
    requests.post(ADD_URL, json={'name': 'foo2', 'task-id': '3'})
    requests.post(ADD_URL, json={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL.format(id=task_id))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 14: ✅ 0.25 points")

def test_15_get_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot read task'

    requests.post(RESET_URL)
    os.remove(STORAGE_FILE)

    # Act
    response = requests.get(GET_URL.format(id='4'))

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 15: ✅ 0.25 points")


test_01_verify_main_page()
test_02_add_task()
test_03_add_task_no_task_id()
test_04_add_task_no_name()
test_05_add_task_nan_task_id()
test_06_add_task_invalid_storage()
test_07_add_task_existing_id()
test_08_remove_task()
test_09_remove_task_nan_task_id()
test_10_remove_task_non_existing_task()
test_11_remove_task_invalid_storage()
test_12_get_task()
test_13_get_task_nan_task_id()
test_14_get_task_non_existing_task()
test_15_get_task_invalid_storage()
